In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from datetime import datetime
import time


import PIL
# Rmb to run "pip install pillow"
# from ResNet import Bottleneck, ResNet, ResNet50

In [2]:
import torch
import torchvision
torch.cuda.is_available()

True

In [3]:
torch.cuda.is_available()

True

In [4]:
import os
from os import listdir

In [5]:
# Folders
train_FAKE_folder_path = "C:/Users/flame/Documents/Optimization Part Small Dataset/Actual Subset/train/fake"
train_REAL_folder_path = "C:/Users/flame/Documents/Optimization Part Small Dataset/Actual Subset/train/real"

test_FAKE_folder_path = "C:/Users/flame/Documents/Optimization Part Small Dataset/Actual Subset/test/fake"
test_REAL_folder_path = "C:/Users/flame/Documents/Optimization Part Small Dataset/Actual Subset/test/real"

In [6]:
def ignore_alpha(image):
    if image.mode == 'RGBA':
        r, g, b, _ = image.split()
        return PIL.Image.merge("RGB", (r, g, b))
    elif image.mode == 'RGB':
        return image
    else:
        return image.convert("RGB")

transform_train_V1 = transforms.Compose([
    ignore_alpha,
    transforms.Resize((224,224)),
    transforms.CenterCrop((224,224)),
    transforms.RandomHorizontalFlip(), # To introduce image orientation variation
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

])

transform_test_V1 = transforms.Compose([
    ignore_alpha,
    transforms.Resize((224, 224)),
    transforms.CenterCrop((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [7]:
train_tensor_list =[]
#Format = list of tuples with each tuple in this format ((tensor version of image), label)

# FAKE portion
for images in os.listdir(train_FAKE_folder_path):
    # print(images)
    # if images.endswith(".png"):
    img = PIL.Image.open(f"{train_FAKE_folder_path}/%s" % images)
    # train_tensor_list.append(transform_train_V1(img))
    train_tensor_list.append((transform_train_V1(img), 0))
# REAL portion
for images in os.listdir(train_REAL_folder_path):
    # if images.endswith(".png"):
    img = PIL.Image.open(f"{train_REAL_folder_path}/%s" % images)
    # train_tensor_list.append(transform_train_V1(img))
    train_tensor_list.append((transform_train_V1(img), 1))

In [8]:
test_tensor_list =[]
#Format = list of tuples with each tuple in this format ((tensor version of image), label)

# FAKE portion
for images in os.listdir(test_FAKE_folder_path):
    # print(images)
    img = PIL.Image.open(f"{test_FAKE_folder_path}/%s" % images)
    # test_tensor_list.append(transform_test_V1(img))
    test_tensor_list.append((transform_test_V1(img),0))
# REAL portion

for images in os.listdir(test_REAL_folder_path):
    # print(images)
    img = PIL.Image.open(f"{test_REAL_folder_path}/%s" % images)
    # test_tensor_list.append(transform_test_V1(img))
    test_tensor_list.append((transform_test_V1(img),1))

In [ ]:
print(len(train_tensor_list))
print(len(test_tensor_list))

32999
12000


In [9]:
print(len(train_tensor_list))
print(len(test_tensor_list))

3300
1200


In [ ]:
A

In [10]:
## LOAD DATA to trainloader and testloader

# train = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)

trainloader = torch.utils.data.DataLoader(train_tensor_list, batch_size=128, shuffle=True, num_workers=2)

# test = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

testloader = torch.utils.data.DataLoader(test_tensor_list, batch_size=128,shuffle=False, num_workers=2)

for X, y in trainloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([128, 3, 224, 224])
Shape of y: torch.Size([128]) torch.int64


## ResNET Model ResNet50

In [11]:
import torch

# input_batch = torch.stack(train_tensor_list)
# print(input_batch)

# device = (
#     "cuda"
#     if torch.cuda.is_available()
#     else "mps"
#     if torch.backends.mps.is_available()
#     else "cpu"
# )

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

# model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=True)
# # or any of these variants
# # model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet169', pretrained=True)
# # model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet201', pretrained=True)
# # model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet161', pretrained=True)
# model.eval()

# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
# or any of these variants
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)
model.eval()

if torch.cuda.is_available():
    model.cuda()

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

# with torch.no_grad():
#     output = model(input_batch)
# # Tensor of shape 1000, with confidence scores over ImageNet's 1000 classes
# print(output[0])
# # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
# probabilities = torch.nn.functional.softmax(output[0], dim=0)
# print(probabilities)

Using cuda device


Using cache found in C:\Users\flame/.cache\torch\hub\pytorch_vision_v0.10.0
C:\Users\flame\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\flame\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [13]:
optimizer_lr =[1e-2,1e-3,1e-4]

for j in optimizer_lr:
    model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
    model.train()
    if torch.cuda.is_available():
        model.cuda()

    
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=j)
    
    def train(dataloader, model, loss_fn, optimizer):
        size = len(dataloader.dataset)
        model.train()
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
    
            # Compute prediction error
            pred = model(X)
            loss = loss_fn(pred, y)
    
            # Backpropagation
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
    
            if batch % 100 == 0:
                loss, current = loss.item(), (batch + 1) * len(X)
                # print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        return loss
    
    def test(dataloader, model, loss_fn):
        size = len(dataloader.dataset)
        num_batches = len(dataloader)
        model.eval()
        test_loss, correct = 0, 0
        with torch.no_grad():
            for X, y in dataloader:
                X, y = X.to(device), y.to(device)
                pred = model(X)
                test_loss += loss_fn(pred, y).item()
                correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= num_batches
        correct /= size
        # print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
        return (100*correct) , test_loss
    
    epochs = 10
    print("Started:", datetime.now())
    
    start = time.time()
    
    for t in range(epochs):
    
        loss = train(trainloader, model, loss_fn, optimizer)
        accuracy , avg_loss = test(testloader, model, loss_fn)
        if t == (epochs-1):
            end = time.time()
            print("Learning Rate:", j)
            print("Start Time :", start)
            print("End Time :", end)
            print("Time Taken:", end - start)
            print(f"Epoch {t+1}\n-------------------------------")
            print("Train Loss :", loss)
            print("Test Avg Loss :", avg_loss)
            print("Test Accuracy :", accuracy)
            print("\n-------------------------------")
print("Done!")

Using cache found in C:\Users\flame/.cache\torch\hub\pytorch_vision_v0.10.0


Started: 2024-04-28 22:08:51.549476
Learning Rate: 0.01
Start Time : 1714313331.5494769
End Time : 1714318764.5401156
Time Taken: 5432.99063873291
Epoch 10
-------------------------------
Train Loss : tensor(0.0090, device='cuda:0', grad_fn=<NllLossBackward0>)
Test Avg Loss : 0.31608742773532866
Test Accuracy : 88.25

-------------------------------


Using cache found in C:\Users\flame/.cache\torch\hub\pytorch_vision_v0.10.0


Started: 2024-04-28 23:39:27.057528
Learning Rate: 0.001
Start Time : 1714318767.0580883
End Time : 1714323912.212061
Time Taken: 5145.153972625732
Epoch 10
-------------------------------
Train Loss : tensor(0.3524, device='cuda:0', grad_fn=<NllLossBackward0>)
Test Avg Loss : 0.42637552618980407
Test Accuracy : 81.16666666666667

-------------------------------


Using cache found in C:\Users\flame/.cache\torch\hub\pytorch_vision_v0.10.0


Started: 2024-04-29 01:05:14.008072
Learning Rate: 0.0001
Start Time : 1714323914.0098786
End Time : 1714329356.2612002
Time Taken: 5442.251321554184
Epoch 10
-------------------------------
Train Loss : tensor(6.3532, device='cuda:0', grad_fn=<NllLossBackward0>)
Test Avg Loss : 7.275066471099853
Test Accuracy : 4.916666666666666

-------------------------------
Done!


## ResNET Model ResNet18

In [14]:
import torch

# input_batch = torch.stack(train_tensor_list)
# print(input_batch)

# device = (
#     "cuda"
#     if torch.cuda.is_available()
#     else "mps"
#     if torch.backends.mps.is_available()
#     else "cpu"
# )

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

# model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=True)
# # or any of these variants
# # model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet169', pretrained=True)
# # model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet201', pretrained=True)
# # model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet161', pretrained=True)
# model.eval()

model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
# or any of these variants
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)
model.eval()

if torch.cuda.is_available():
    model.cuda()

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

# with torch.no_grad():
#     output = model(input_batch)
# # Tensor of shape 1000, with confidence scores over ImageNet's 1000 classes
# print(output[0])
# # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
# probabilities = torch.nn.functional.softmax(output[0], dim=0)
# print(probabilities)

Using cuda device


Using cache found in C:\Users\flame/.cache\torch\hub\pytorch_vision_v0.10.0
C:\Users\flame\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\flame/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|█████████████████████████████████████████████████████████████████████████████| 44.7M/44.7M [00:01<00:00, 27.8MB/s]


In [15]:
optimizer_lr =[1e-2,1e-3,1e-4]

for j in optimizer_lr:
    model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
    model.train()
    if torch.cuda.is_available():
        model.cuda()

    
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=j)
    
    def train(dataloader, model, loss_fn, optimizer):
        size = len(dataloader.dataset)
        model.train()
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
    
            # Compute prediction error
            pred = model(X)
            loss = loss_fn(pred, y)
    
            # Backpropagation
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
    
            if batch % 100 == 0:
                loss, current = loss.item(), (batch + 1) * len(X)
                # print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        return loss
    
    def test(dataloader, model, loss_fn):
        size = len(dataloader.dataset)
        num_batches = len(dataloader)
        model.eval()
        test_loss, correct = 0, 0
        with torch.no_grad():
            for X, y in dataloader:
                X, y = X.to(device), y.to(device)
                pred = model(X)
                test_loss += loss_fn(pred, y).item()
                correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= num_batches
        correct /= size
        # print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
        return (100*correct) , test_loss
    
    epochs = 10
    print("Started:", datetime.now())
    
    start = time.time()
    
    for t in range(epochs):
    
        loss = train(trainloader, model, loss_fn, optimizer)
        accuracy , avg_loss = test(testloader, model, loss_fn)
        if t == (epochs-1):
            end = time.time()
            print("Learning Rate:", j)
            print("Start Time :", start)
            print("End Time :", end)
            print("Time Taken:", end - start)
            print(f"Epoch {t+1}\n-------------------------------")
            print("Train Loss :", loss)
            print("Test Avg Loss :", avg_loss)
            print("Test Accuracy :", accuracy)
            print("\n-------------------------------")
print("Done!")

Using cache found in C:\Users\flame/.cache\torch\hub\pytorch_vision_v0.10.0


Started: 2024-04-29 02:36:00.624125
Learning Rate: 0.01
Start Time : 1714329360.63976
End Time : 1714331166.7576187
Time Taken: 1806.1178586483002
Epoch 10
-------------------------------
Train Loss : tensor(0.0318, device='cuda:0', grad_fn=<NllLossBackward0>)
Test Avg Loss : 0.46994025111198423
Test Accuracy : 83.5

-------------------------------


Using cache found in C:\Users\flame/.cache\torch\hub\pytorch_vision_v0.10.0


Started: 2024-04-29 03:06:07.336793
Learning Rate: 0.001
Start Time : 1714331167.3367932
End Time : 1714332974.5590918
Time Taken: 1807.2222986221313
Epoch 10
-------------------------------
Train Loss : tensor(0.5867, device='cuda:0', grad_fn=<NllLossBackward0>)
Test Avg Loss : 0.5930582046508789
Test Accuracy : 74.75

-------------------------------


Using cache found in C:\Users\flame/.cache\torch\hub\pytorch_vision_v0.10.0


Started: 2024-04-29 03:36:15.159233
Learning Rate: 0.0001
Start Time : 1714332975.1592333
End Time : 1714334767.776856
Time Taken: 1792.6176226139069
Epoch 10
-------------------------------
Train Loss : tensor(10.0824, device='cuda:0', grad_fn=<NllLossBackward0>)
Test Avg Loss : 9.837989997863769
Test Accuracy : 1.6666666666666667

-------------------------------
Done!


## ResNET Model ResNet101

In [16]:
import torch

# input_batch = torch.stack(train_tensor_list)
# print(input_batch)

# device = (
#     "cuda"
#     if torch.cuda.is_available()
#     else "mps"
#     if torch.backends.mps.is_available()
#     else "cpu"
# )

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

# model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=True)
# # or any of these variants
# # model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet169', pretrained=True)
# # model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet201', pretrained=True)
# # model = torch.hub.load('pytorch/vision:v0.10.0', 'densenet161', pretrained=True)
# model.eval()

# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
# or any of these variants
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)
model.eval()

if torch.cuda.is_available():
    model.cuda()

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

# with torch.no_grad():
#     output = model(input_batch)
# # Tensor of shape 1000, with confidence scores over ImageNet's 1000 classes
# print(output[0])
# # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
# probabilities = torch.nn.functional.softmax(output[0], dim=0)
# print(probabilities)

Using cuda device


Using cache found in C:\Users\flame/.cache\torch\hub\pytorch_vision_v0.10.0
C:\Users\flame\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to C:\Users\flame/.cache\torch\hub\checkpoints\resnet101-63fe2227.pth


URLError: <urlopen error [Errno 11001] getaddrinfo failed>

In [ ]:
optimizer_lr =[1e-2,1e-3,1e-4]

for j in optimizer_lr:
    model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
    model.train()
    if torch.cuda.is_available():
        model.cuda()

    
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=j)
    
    def train(dataloader, model, loss_fn, optimizer):
        size = len(dataloader.dataset)
        model.train()
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
    
            # Compute prediction error
            pred = model(X)
            loss = loss_fn(pred, y)
    
            # Backpropagation
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
    
            if batch % 100 == 0:
                loss, current = loss.item(), (batch + 1) * len(X)
                # print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        return loss
    
    def test(dataloader, model, loss_fn):
        size = len(dataloader.dataset)
        num_batches = len(dataloader)
        model.eval()
        test_loss, correct = 0, 0
        with torch.no_grad():
            for X, y in dataloader:
                X, y = X.to(device), y.to(device)
                pred = model(X)
                test_loss += loss_fn(pred, y).item()
                correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= num_batches
        correct /= size
        # print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
        return (100*correct) , test_loss
    
    epochs = 10
    print("Started:", datetime.now())
    
    start = time.time()
    
    for t in range(epochs):
    
        loss = train(trainloader, model, loss_fn, optimizer)
        accuracy , avg_loss = test(testloader, model, loss_fn)
        if t == (epochs-1):
            end = time.time()
            print("Learning Rate:", j)
            print("Start Time :", start)
            print("End Time :", end)
            print("Time Taken:", end - start)
            print(f"Epoch {t+1}\n-------------------------------")
            print("Train Loss :", loss)
            print("Test Avg Loss :", avg_loss)
            print("Test Accuracy :", accuracy)
            print("\n-------------------------------")
print("Done!")

In [ ]:
epochs = 10
print("Started:", datetime.now())
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(testloader, model, loss_fn)
print("Done!")

Started: 2024-04-11 07:46:36.077717
Epoch 1
-------------------------------
loss: 16.818741  [  128/32999]
loss: 0.685726  [12928/32999]


In [ ]:
epochs = 10
print("Started:", datetime.now())
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(testloader, model, loss_fn)
print("Done!")

Started: 2024-04-12 01:40:54.958049
Epoch 1
-------------------------------
loss: 16.974977  [  128/32999]
loss: 0.680214  [12928/32999]
loss: 0.469419  [25728/32999]
Test Error: 
 Accuracy: 84.5%, Avg loss: 0.374817 

Epoch 2
-------------------------------
loss: 0.386840  [  128/32999]
loss: 0.375520  [12928/32999]
loss: 0.258455  [25728/32999]
Test Error: 
 Accuracy: 89.1%, Avg loss: 0.271099 

Epoch 3
-------------------------------
loss: 0.249269  [  128/32999]
loss: 0.259741  [12928/32999]
loss: 0.251562  [25728/32999]
Test Error: 
 Accuracy: 91.5%, Avg loss: 0.220083 

Epoch 4
-------------------------------
loss: 0.145130  [  128/32999]
loss: 0.195809  [12928/32999]
loss: 0.166314  [25728/32999]
Test Error: 
 Accuracy: 92.4%, Avg loss: 0.196617 

Epoch 5
-------------------------------
loss: 0.134833  [  128/32999]
loss: 0.141006  [12928/32999]
loss: 0.146252  [25728/32999]
Test Error: 
 Accuracy: 92.8%, Avg loss: 0.183113 

Epoch 6
-------------------------------
loss: 0.12509

In [ ]:
# Save model

print("Finished:", datetime.now())
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Finished: 2024-04-12 21:19:00.036260
Saved PyTorch Model State to model.pth


**bold text**### Final Evaluation Metrics (f1 Score and Confusion Matrix)

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

In [ ]:
def final_evaluation(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    pred_list =[]
    true_list=[]


    with torch.no_grad():
        for X, y in dataloader:

            X, y = X.to(device), y.to(device)
            pred = model(X)

            true_list.extend(y.cpu().tolist())
            pred_list.extend(pred.argmax(1).cpu().tolist())

            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size

    f1_result = f1_score(true_list, pred_list)
    tn, fp, fn, tp = confusion_matrix(true_list, pred_list).ravel()
    confusion_accuracy = (tp+tn)/(tn+fp+fn+tp)
    confusion_recall = tp/(tp+fn)
    confusion_precision = tp/(tp+fp)

    print(f"Confusion Matrix: \n Accuracy: {(100*confusion_accuracy):>0.1f}% \n Recall: {(100*confusion_recall):>0.1f}% \n Precision: {(100*confusion_precision):>0.1f}% \n")
    print(f"F1 Result: {f1_result} \n")
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
final_evaluation(testloader, model, loss_fn)

Confusion Matrix: 
 Accuracy: 94.8% 
 Recall: 94.4% 
 Precision: 95.1% 

F1 Result: 0.9477118714966953 

Test Error: 
 Accuracy: 94.8%, Avg loss: 0.135091 



## Test Load Model from external local file and rerun the Final Evaluation

In [ ]:
base_model_test = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)

In [ ]:
# if torch.cuda.is_available():
#     base_model_test.cuda()

# final_evaluation(testloader, base_model_test, loss_fn)
# #Returns error if just straight run base model

In [ ]:
path = "ResNet50-MidJourney-Big-DataSet-model.pth"

base_model_test.load_state_dict(torch.load(path))

<All keys matched successfully>

In [ ]:
final_evaluation(testloader, base_model_test, loss_fn)

## IGNORE

In [ ]:
# def final_evaluation(dataloader, model, loss_fn):
#     size = len(dataloader.dataset)
#     num_batches = len(dataloader)
#     model.eval()
#     test_loss, correct = 0, 0
#     pred_list =[]
#     true_list=[]


#     with torch.no_grad():
#         for X, y in dataloader:
#             # print("X.shape :",X.shape)
#             # print("y.shape :",y.shape)
#             # # X.shape : torch.Size([128, 3, 224, 224])
#             # # y.shape : torch.Size([128])

#             X, y = X.to(device), y.to(device)
#             pred = model(X)
#             # print("pred.argmax(1).shape :", pred.argmax(1).shape)
#             # print("pred.argmax(1) :", pred.argmax(1))
#             # # pred.argmax(1).shape : torch.Size([128])
#             # # pred.argmax(1) : tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#             # #         0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#             # #         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#             # #         0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#             # #         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#             # #         0, 0, 0, 0, 0, 0, 1, 0], device='cuda:0')

#             true_list.extend(y.cpu().tolist())
#             pred_list.extend(pred.argmax(1).cpu().tolist())
#             # print("true_list[0].shape :",true_list[0].shape)
#             # print("pred_list[0].shape :",pred_list[0].shape)
#             # # true_list[0].shape : torch.Size([128])
#             # # pred_list[0].shape : torch.Size([128, 1000])

#             # pred.argmax(1)
#             test_loss += loss_fn(pred, y).item()
#             correct += (pred.argmax(1) == y).type(torch.float).sum().item()
#     test_loss /= num_batches
#     correct /= size

#     f1_result = f1_score(true_list, pred_list)
#     tn, fp, fn, tp = confusion_matrix(true_list, pred_list).ravel()
#     confusion_accuracy = (tp+tn)/(tn+fp+fn+tp)
#     confusion_recall = tp/(tp+fn)
#     confusion_precision = tp/(tp+fp)

#     print(f"Confusion Matrix: \n Accuracy: {(100*confusion_accuracy):>0.1f}% \n Recall: {(100*confusion_recall):>0.1f}% \n Precision: {(100*confusion_precision):>0.1f}% \n")
#     print(f"F1 Result: {f1_result} \n")
#     print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")